In [3]:
from pathlib import Path
"Module" in str(Path.cwd())

True

In [4]:
from XQuant import Tools
Tools.get_newest_file("ResConSecTarpriScore")

'E:\\Share\\Stk_Data\\dataFile\\ResConSecTarpriScore\\ResConSecTarpriScore_Y2023.h5'

In [1]:
from XQuant import get_data
get_data("ACCA_gm", begin='20220101')

,symbol,end_date,MANANETR,NETPROFIT,TOTASSET,pub_date
0,SHSE.600000,2021-12-31,-2.571920e+11,5.376600e+10,8.136757e+12,2022-04-28
1,SHSE.600000,2022-06-30,6.525700e+10,3.059600e+10,8.505959e+12,2022-08-27
2,SHSE.600004,2021-12-31,8.448503e+08,-3.854062e+08,2.754503e+10,2022-04-09
3,SHSE.600004,2022-06-30,1.668371e+08,-5.306237e+08,2.767428e+10,2022-08-31
4,SHSE.600006,2021-12-31,9.803643e+08,3.822105e+08,1.988165e+10,2022-03-31
...,...,...,...,...,...,...
11134,SZSE.301369,2022-12-31,1.438536e+08,1.264835e+08,1.705313e+09,2023-03-20
11135,SZSE.301377,2022-12-31,1.561280e+08,2.226261e+08,2.935961e+09,2023-04-15
11136,SZSE.301380,2022-12-31,2.478149e+07,5.870761e+07,1.026831e+09,2023-03-20
11137,SZSE.301389,2022-12-31,1.850968e+08,1.688592e+08,2.403445e+09,2023-03-24
